<a href="https://colab.research.google.com/github/yogesh-bhattarai/GEN_AI/blob/main/exercise_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import pandas as pd
import numpy as np

In [4]:
df= pd.read_csv("/content/ExerciseRecommendation.csv")

In [8]:
df.head(10)

,exercise_id,fitness_goal,exercise_name,exercise_type,equipment_needed,duration_or_reps
0,1,Weight Loss,Jump rope,Cardio,Jump rope,3 sets of 15 reps
1,2,Weight Loss,Burpees,Cardio,NaN,3 sets of 15 reps
2,3,Weight Loss,HIIT,Cardio,Mat,3 sets of 15 reps
3,4,Weight Loss,Running,Cardio,Treadmill / Outdoor,20-40 mins
4,5,Weight Loss,Cycling,Cardio,Stationary bike,20-40 mins
5,6,Weight Loss,Mountain climbers,Cardio,Mat,3 sets of 15 reps
6,7,Weight Loss,Squats,Strength,None / Barbell,3 sets of 8-12 reps
7,8,Muscle Gain,Bench press,Strength,Barbell,3 sets of 8-12 reps
8,9,Muscle Gain,Deadlifts,Strength,Barbell,3 sets of 8-12 reps
9,10,Muscle Gain,Squats,Strength,None / Barbell,3 sets of 8-12 reps


In [10]:
# Step 2: Preprocessing (Label Encoding for categorical features)
le_goal = LabelEncoder()
le_type = LabelEncoder()
le_equipment = LabelEncoder()

In [13]:
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import NearestNeighbors


In [11]:
df['fitness_goal_encoded'] = le_goal.fit_transform(df['fitness_goal'])
df['exercise_type_encoded'] = le_type.fit_transform(df['exercise_type'])
df['equipment_encoded'] = le_equipment.fit_transform(df['equipment_needed'])

In [12]:
features = df[['fitness_goal_encoded', 'exercise_type_encoded', 'equipment_encoded', 'duration_or_reps']]


In [15]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import NearestNeighbors
def extract_numeric_reps(value):
    try:
        return int(value.split('of')[-1].split('reps')[0].strip())
    except (ValueError, IndexError, AttributeError):
        return None

def extract_numeric_duration(value):
    """Extracts numeric duration from a string (e.g., '30 minutes' -> 30)."""
    try:
        return int(value.split('minutes')[0].strip())
    except (ValueError, IndexError, AttributeError):
        return None
df['reps'] = df['duration_or_reps'].apply(extract_numeric_reps)
df['duration'] = df['duration_or_reps'].apply(extract_numeric_duration)

df['reps'] = df['reps'].fillna(0)
df['duration'] = df['duration'].fillna(0)
features = df[['fitness_goal_encoded', 'exercise_type_encoded', 'equipment_encoded', 'reps', 'duration']]

knn = NearestNeighbors(n_neighbors=3, algorithm='auto')
knn.fit(features)

<ipython-input-15-3fe517db9c12>:30: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['duration'] = df['duration'].fillna(0)


NearestNeighbors(n_neighbors=3)

In [26]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import NearestNeighbors

# ... (your existing code for data loading and preprocessing) ...

def recommend_exercise(goal, ex_type, equipment, duration):
    # Handle unseen labels for 'fitness_goal'
    if goal not in le_goal.classes_:
        le_goal.classes_ = np.append(le_goal.classes_, goal)  # Add the new label

    # Handle unseen labels for 'exercise_type'
    if ex_type not in le_type.classes_:
        le_type.classes_ = np.append(le_type.classes_, ex_type)  # Add the new label

    # Handle unseen labels for 'equipment'
    if equipment not in le_equipment.classes_:
        le_equipment.classes_ = np.append(le_equipment.classes_, equipment)  # Add the new label

    input_data = pd.DataFrame({
        'fitness_goal_encoded': [le_goal.transform([goal])[0]],
        'exercise_type_encoded': [le_type.transform([ex_type])[0]],
        'equipment_encoded': [le_equipment.transform([equipment])[0]],
        'duration': [duration]  # Assuming 'duration' is the intended feature
    })

    # Calculate distances and indices within the function
    distances, indices = knn.kneighbors(input_data)

    print("Recommended Exercises:")
    for idx in indices[0]:
        print(f"- {df.loc[idx, 'exercise_name']} ({df.loc[idx, 'exercise_type']}, {df.loc[idx, 'duration']} mins)")

In [27]:
recommend_exercise('Strength Building', 'Weight Training', 'Dumbbells', 30)

ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- reps


In [20]:
recommend_exercise('Strength Building', 'Weight Training', 'Dumbbells', 30)


ValueError: y contains previously unseen labels: 'Strength Building'